In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# read p10 spatial data
p10 = gpd.read_file(r'M:\Data\GIS layers\UrbanSim smelt\p10_EIR\p10_parcels.shp')

p10_df = p10[['PARCEL_ID', 'geometry']]
p10_df['PARCEL_ID'] = p10_df['PARCEL_ID'].apply(lambda x: int(round(x)))
print('read {} rows of p10 spatial data'.format(p10_df.shape[0]))
print(p10_df.dtypes)

read 1956207 rows of p10 spatial data
PARCEL_ID       int64
geometry     geometry
dtype: object


C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [15]:
# read parcel_geographic data with 'fbpchcat' label for parcels
pz = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Current PBA50 Large General Input Data\2021_02_25_parcels_geography.csv',
                 usecols = ['PARCEL_ID', 'fbp_gg_id', 'hra_id', 'fbp_tra_id'])
print('read {} rows of parcel_geography data'.format(pz.shape[0]))
print(pz.dtypes)

# join fbpchcat to parcel spatial data
p10_z = p10_df.merge(pz, on='PARCEL_ID', how='left')
display(p10_z.head())

# consolidate categories
tra_dict = {'tra3' : 'tra3',
            'tra2c': 'tra2', 
            'tra2a': 'tra2', 
            'tra1' : 'tra1', 
            'tra2b': 'tra2'}
p10_z['fbp_tra_id'] = p10_z['fbp_tra_id'].map(tra_dict)

# fill NAs and create growth geography categories
p10_z['fbp_gg_id'].fillna('NonGG', inplace=True)
p10_z['hra_id'].fillna('NonHRA', inplace=True)
p10_z['fbp_tra_id'].fillna('NonTRA', inplace=True)

p10_z['gg_cat'] = p10_z['fbp_gg_id'] + '/' + p10_z['hra_id'] + '/' + p10_z['fbp_tra_id']
print('unique gg_cat:')
print(p10_z.gg_cat.unique())

read 1956208 rows of parcel_geography data
PARCEL_ID      int64
hra_id        object
fbp_gg_id     object
fbp_tra_id    object
dtype: object


,PARCEL_ID,geometry,hra_id,fbp_gg_id,fbp_tra_id
0,229116,"POLYGON ((605993.404 4168316.032, 605993.297 4...",HRA,NaN,NaN
1,244166,"POLYGON ((613491.065 4175262.642, 613490.995 4...",NaN,NaN,tra3
2,202378,"POLYGON ((586745.708 4168287.089, 586746.546 4...",NaN,NaN,NaN
3,2004420,"POLYGON ((520226.869 4286274.188, 520222.958 4...",NaN,NaN,NaN
4,340332,"POLYGON ((590655.279 4155999.718, 590590.988 4...",HRA,NaN,NaN


unique gg_cat:
['NonGG/HRA/NonTRA' 'NonGG/NonHRA/tra3' 'NonGG/NonHRA/NonTRA'
 'NonGG/HRA/tra3' 'GG/HRA/NonTRA' 'GG/NonHRA/tra2' 'GG/HRA/tra2'
 'GG/HRA/tra3' 'GG/NonHRA/NonTRA' 'GG/NonHRA/tra3' 'GG/HRA/tra1'
 'NonGG/HRA/tra2' 'NonGG/NonHRA/tra2' 'GG/NonHRA/tra1' 'NonGG/HRA/tra1'
 'NonGG/NonHRA/tra1']


In [16]:
# select GG parcels and export
p10_gg = p10_z.loc[p10_z.fbp_gg_id == 'GG']
print(p10_gg.gg_cat.unique())
print('export {} rows of p10_fbp_gg'.format(p10_gg.shape[0]))
p10_gg.to_file(r'M:\Data\GIS layers\Blueprint Land Use Strategies\FBP_GG\p10_fbp_gg_categories.shp')

['GG/HRA/NonTRA' 'GG/NonHRA/tra2' 'GG/HRA/tra2' 'GG/HRA/tra3'
 'GG/NonHRA/NonTRA' 'GG/NonHRA/tra3' 'GG/HRA/tra1' 'GG/NonHRA/tra1']
export 543921 rows of p10_fbp_gg
